In [1]:
import pandas as pd
from selenium.webdriver.common.keys import Keys 
from selenium.webdriver.chrome.options import Options

# pip install webdriver-manager
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import numpy as np
from selenium.webdriver.common.action_chains import ActionChains
import pickle
import time 

import re

import urllib.request
import os
from bs4 import BeautifulSoup
from tqdm import tqdm
import numpy as np
from time import sleep
import argparse
import shutil
import pandas as pd
import time
from selenium.webdriver.common.keys import Keys
import urllib.request
import glob

import urllib.request


In [2]:
s=Service(ChromeDriverManager().install())

option = Options()
driver = webdriver.Chrome(service=s, chrome_options=option)
driver.create_options()

driver.maximize_window()

C:\Users\ENG~1.MOU\AppData\Local\Temp/ipykernel_5236/388229857.py:4: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(service=s, chrome_options=option)


# Functions

In [3]:
wait = WebDriverWait(driver,10)

In [4]:
def found():
    global x
    number_of_ads = wait.until(EC.visibility_of_element_located((By.XPATH,"//div[@class='_76047990']")))
    txt = number_of_ads.text
    print("Hi I found " + txt)
    
    num = txt.split(" ")
    x = int(float(num[0].replace(',', '.')))


In [5]:
def not_found():
    no_results = wait.until(EC.visibility_of_element_located((By.TAG_NAME,"h3")))
    print(no_results.text)
    driver.quit()

In [6]:
def scrape_or_not():
    print("1 ==> Scrape all products ")
    print("2 ==> Scrape products by number")
    print("3 ==> close the page ")
    
    choice = int(input())
    if choice == 1:
        scrape_all(x)
    elif choice == 2:
        scrape_by_num()
    elif choice == 3:
        driver.close()


In [7]:
def search_something():
    text = input("What do you want to search about")
    search_element = wait.until(EC.visibility_of_element_located((By.XPATH,"//input[@type='search']")))
    search_element.send_keys(text)
    
    button = wait.until(EC.visibility_of_element_located((By.CLASS_NAME, "a3e390b5")))
    button.click()
    
    time.sleep(3)
    
    try:
        found()
        scrape_or_not()
    except:
        not_found()

In [8]:
def scrape_all(number):
    try:
        wait.until(EC.visibility_of_element_located((By.XPATH,"//div[@title='Next']")))
        
        while continue_scraping(number):
            get_names(number)
            get_prices(number)
            get_links(number)
            get_infos(number)
            get_pictures(number)
            if len(links) != number:
                driver.execute_script("window.scrollTo(0,8000)")
                click_button = wait.until(EC.visibility_of_element_located((By.XPATH,"//div[@title='Next']")))        
                click_button.click() 
            else:
                break
    except:
            get_names(number)
            get_prices(number)
            get_links(number)
            get_infos(number)
            get_pictures(number)
            
            
            print("finish")
            time.sleep(2)
            driver.close()

In [9]:
def scrape_by_num():
    number = int(input("How much elemnts do you want to scrap?"))
    try:
        wait.until(EC.visibility_of_element_located((By.XPATH,"//div[@title='Next']")))
        
        while continue_scraping(number):
            get_names(number)
            get_prices(number)
            get_links(number)
            get_infos(number)
            get_pictures(number)
            if len(links) != number:
                driver.execute_script("window.scrollTo(0,8000)")
                click_button = wait.until(EC.visibility_of_element_located((By.XPATH,"//div[@title='Next']")))        
                click_button.click() 
            else:
                break
    except:
            get_names(number)
            get_prices(number)
            get_links(number)
            get_infos(number)
            get_pictures(number)
            
            
            print("finish")
            time.sleep(2)
            driver.close()

In [10]:
def continue_scraping(number):
    if len(links) < number:
        return True
    else:
        return False

# Gets

In [11]:
prices = []

In [12]:
def get_prices(number):
    
    above_prices_class_in_page = wait.until(EC.visibility_of_all_elements_located((By.CSS_SELECTOR, "div._41d2b9f3")))
    
    for price in range(len(above_prices_class_in_page)):
        if (number == 0 or len(prices) < number ):
            try :
                above_prices_class_in_page[price].find_element(By.CSS_SELECTOR, "div._52497c97")
                prices.append(above_prices_class_in_page[price].find_element(By.CSS_SELECTOR, "div._52497c97").text)
            except:
                prices.append('Null')
        else:
            break
#     for ele in prices:
#         if ele == "NEGOTIABLE":
#             prices.remove(ele)
#         else:
#             pass


In [13]:
names = []

In [14]:
def get_names(number):
    all_names_in_page = wait.until(EC.visibility_of_all_elements_located((By.CSS_SELECTOR, "article div.ee2b0479 a")))
    
    for i in range(len(all_names_in_page)):
        if (number == 0 or len(names) < number ):
            names.append(all_names_in_page[i].get_attribute("title")) 
        else:
            break
    

In [15]:
links = []

In [16]:
def get_links(number):
    global links
    all_links_in_page = wait.until(EC.visibility_of_all_elements_located((By.CSS_SELECTOR, "article div.ee2b0479 a")))
    
    for i in range(len(all_links_in_page)):
        if (number == 0 or len(links) < number ):
            links.append(all_links_in_page[i].get_attribute("href")) 
        else:
            break

In [17]:
infos = []

In [18]:
def get_infos(number):
    all_infos_in_page = wait.until(EC.visibility_of_all_elements_located((By.CSS_SELECTOR, "div.a52608cc div._4dbba078")))
    
    for i in range(len(all_infos_in_page)):
        if (number == 0 or len(infos) < number ):

            try :       
                all_infos_in_page[i].find_element(By.CSS_SELECTOR, "div.a52608cc div._4dbba078 div.a8f6df88").text + " ... " + all_infos_in_page[i].find_element(By.CSS_SELECTOR, "div.a52608cc div._4dbba078 div._2fc90438").text
                infos.append(all_infos_in_page[i].find_element(By.CSS_SELECTOR, "div.a52608cc div._4dbba078 div.a8f6df88").text + " ... " + all_infos_in_page[i].find_element(By.CSS_SELECTOR, "div.a52608cc div._4dbba078 div._2fc90438").text)
            except:
                infos.append('Null')
        else:
            break

In [19]:
pictures = []

In [20]:
def get_pictures(number):
    all_pictures_in_page = wait.until(EC.visibility_of_all_elements_located((By.CSS_SELECTOR, "div.ee2b0479")))
    
    for i in range(len(all_pictures_in_page)):
        if (number == 0 or len(pictures) < number ):

            try :       
                all_pictures_in_page[i].find_element(By.TAG_NAME,"picture").find_element(By.TAG_NAME, "img")
                pictures.append(all_pictures_in_page[i].find_element(By.TAG_NAME,"picture").find_element(By.TAG_NAME, "img").get_attribute("data-src"))
            except:
                pictures.append('Null')
        else:
            break

# Gooooooooooooooooooooooooooooo

In [21]:
driver.get("https://www.olx.com.eg/en/")

In [22]:
search_something()

What do you want to search aboutdayun
Hi I found 1,232 ads
1 ==> Scrape all products 
2 ==> Scrape products by number
3 ==> close the page 
2
How much elemnts do you want to scrap?250


In [23]:
[len(links),
len(pictures),
len(prices),
len(infos), 
len(names)]

[250, 250, 250, 250, 250]

In [24]:
df = pd.DataFrame({
    "names":names,
    "links" : links,
    "prices":prices,
    "infos":infos,
    "pictures": pictures
    
})

In [25]:
df

,names,links,prices,infos,pictures
0,Dayun max 250 cc,https://www.olx.com.eg/en/ad/dayun-max-250-cc-...,"EGP 23,000","NEW\n•\n2021 ... Mokattam, Cairo•17 hours ago",https://images.olx.com.eg/thumbnails/35636948-...
1,Dayun 6,https://www.olx.com.eg/en/ad/dayun-6-ID1929156...,"EGP 15,000","50000 to 59999 km\n•\n2018 ... Laurent, Alexan...",https://images.olx.com.eg/thumbnails/35559948-...
2,dayun 2b,https://www.olx.com.eg/en/ad/dayun-2b-ID192889...,"EGP 9,000","100 to 9999 km\n•\n2018 ... Dar al-Salaam, Cai...",https://images.olx.com.eg/thumbnails/35424012-...
3,Dayun ktx 250 داوين تريلا ٢٥٠,https://www.olx.com.eg/en/ad/dayun-ktx-250-%D8...,"EGP 22,000","100 to 9999 km\n•\n2021 ... Haram, Giza•5 days...",https://images.olx.com.eg/thumbnails/35235013-...
4,دايو ٦ للبيع dayun 6,https://www.olx.com.eg/en/ad/%D8%AF%D8%A7%D9%8...,"EGP 18,500","100 to 9999 km\n•\n2021 ... Matareya, Cairo•1 ...",https://images.olx.com.eg/thumbnails/34879617-...
...,...,...,...,...,...
245,تروسيكل للبيع,https://www.olx.com.eg/en/ad/%D8%AA%D8%B1%D9%8...,"EGP 17,000","0 to 99 km\n•\n2020 ... Shubra al-Khaimah, Qal...",https://images.olx.com.eg/thumbnails/30708753-...
246,موتوسيكل,https://www.olx.com.eg/en/ad/%D9%85%D9%88%D8%A...,"EGP 16,300","NEW\n•\n2022 ... Samanoud, Gharbia•2 months ago",https://images.olx.com.eg/thumbnails/30707703-...
247,مكنه هوجن وينج 2012,https://www.olx.com.eg/en/ad/%D9%85%D9%83%D9%8...,"EGP 6,000","0 to 99 km\n•\n2012 ... Seyouf, Alexandria•2 m...",https://images.olx.com.eg/thumbnails/30704040-...
248,دايو ٤,https://www.olx.com.eg/en/ad/%D8%AF%D8%A7%D9%8...,"EGP 16,000","100 to 9999 km\n•\n2022 ... Arbaeen, Suez•2 mo...",https://images.olx.com.eg/thumbnails/30700234-...


In [26]:
path = os.path.join(os.getcwd(), "df.csv")
df.to_csv(path_or_buf=path, encoding = 'utf-8')